In [1]:
#importing required libraries
from keras.datasets import cifar100
from tensorflow import keras
import keras, os
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, AvgPool2D
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Input, Softmax
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.metrics import Precision, Recall
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras import backend as K
from keras import regularizers

In [2]:
train=True
num_classes = 100

In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train=x_train/255.0
x_test=x_test/255.0

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

169009152/169001437 [==============================] - 5s 0us/step


In [4]:
#Model summary using the VGG16 archtecture
x_shape = keras.layers.Input(shape = (32,32,3))
model = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x_shape)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2,2),strides=(2,2))(model)

model = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2,2),strides=(2,2))(model)

model = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2,2),strides=(2,2))(model)

model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2,2),strides=(2,2))(model)

model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2,2),strides=(2,2))(model)

model = Flatten()(model)
model = Dense(units=2048, activation="relu")(model)
model = Dropout(0.3)(model)
model = Dense(units=1024,activation="relu")(model)
model = Dropout(0.3)(model)
model = Dense(units=100,activation="softmax")(model)

model1 = Model(x_shape,model)
model1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)       

In [5]:
#optimization details
opt = Adam(lr=0.0001, decay = 1e-06)
model1.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

In [12]:
batch_size = 128

checkpoint = ModelCheckpoint("vggdropouts.h5", monitor='val_accuracy', verbose=2, save_best_only=True, save_weights_only=True, mode='auto')
early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=30, verbose=2, mode='auto')


datagen = ImageDataGenerator(width_shift_range = 0.1,
                             height_shift_range = 0.1,
                             horizontal_flip = True,
                             vertical_flip = False,
                             rotation_range = 10,
                             fill_mode = 'nearest')
# datagen.fit(x_train)


hist = model1.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=50,
                    validation_data=(x_test, y_test),callbacks=[checkpoint, early],verbose=2)



Epoch 1/50

Epoch 00001: val_accuracy improved from -inf to 0.60410, saving model to vggdropouts.h5
390/390 - 30s - loss: 0.3446 - accuracy: 0.8921 - precision: 0.9184 - recall: 0.8672 - val_loss: 2.1875 - val_accuracy: 0.6041 - val_precision: 0.6521 - val_recall: 0.5840
Epoch 2/50

Epoch 00002: val_accuracy improved from 0.60410 to 0.61710, saving model to vggdropouts.h5
390/390 - 30s - loss: 0.3354 - accuracy: 0.8934 - precision: 0.9196 - recall: 0.8704 - val_loss: 2.1036 - val_accuracy: 0.6171 - val_precision: 0.6625 - val_recall: 0.5960
Epoch 3/50

Epoch 00003: val_accuracy did not improve from 0.61710
390/390 - 30s - loss: 0.3285 - accuracy: 0.8949 - precision: 0.9195 - recall: 0.8725 - val_loss: 2.1909 - val_accuracy: 0.6064 - val_precision: 0.6530 - val_recall: 0.5882
Epoch 4/50

Epoch 00004: val_accuracy did not improve from 0.61710
390/390 - 29s - loss: 0.3262 - accuracy: 0.8955 - precision: 0.9207 - recall: 0.8732 - val_loss: 2.1724 - val_accuracy: 0.6092 - val_precision: 0.6

In [17]:
model1.save_weights('vggdropouts.h5') 

In [ ]:
model1.load_weights(filepath = 'vggdropouts.h5')
scores = model1.evaluate(x_train, y_train, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Precison: %.2f%%" % (scores[2]*100))
print("Recall: %.2f%%" % (scores[3]*100))

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.legend(['train','test'])
plt.title('loss')
plt.figure()
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.legend(['train','test'])
plt.title('accuracy')
plt.figure()
plt.plot(hist.history['precision'])
plt.plot(hist.history['val_precision'])
plt.legend(['train','test'])
plt.title('precision')
plt.figure()
plt.plot(hist.history['recall'])
plt.plot(hist.history['val_recall'])
plt.legend(['train','test'])
plt.title('recall')

In [21]:
# Printing the saved weights in numpy arrays
for layer in model1.layers: 
  print(layer.get_weights())

Streaming output truncated to the last 5000 lines.
       -0.00254356, -0.00699103,  0.16334057,  0.07255666,  0.07113353,
       -0.05390637,  0.13512893,  0.0125494 , -0.07360926, -0.13685723,
        0.01869873, -0.08053155,  0.0985668 ,  0.09475082,  0.16258897,
        0.10399104,  0.09424368, -0.05177534, -0.13037004, -0.00925699,
        0.02028279,  0.06506288, -0.00502293,  0.12507553, -0.07791988,
        0.06401544, -0.18538696, -0.11768304,  0.09151175,  0.1656384 ,
        0.04391866, -0.01529799,  0.13306881,  0.07706137,  0.06341037,
        0.14899391,  0.10085048,  0.10636205,  0.11351751,  0.15043743,
        0.21163312, -0.00575752,  0.06563685,  0.16303961,  0.1743241 ,
        0.16059673,  0.11470842,  0.00066418,  0.07732595,  0.1315957 ,
        0.09911501,  0.03093379,  0.0658195 ,  0.03141638,  0.02527587,
        0.00237361,  0.11795219,  0.13108803,  0.10263592, -0.059176  ,
        0.148915  ,  0.10013796,  0.00193508,  0.13690737,  0.04938282,
        0.150